(sec:T3:SF:convergencia)=
### Convergencia de la serie de Fourier

La serie de Fourier proporciona una representación adecuada de una señal periódica siempre que ésta tenga **potencia finita**. En particular, para una señal periódica $\tilde{x}(t)$ de período fundamental $T_0$:

:::{important .simple icon=false} Potencia finita
```{math}
	P_\infty = \frac{1}{T_0}\int_{\langle T_0 \rangle} |\tilde{x}(t)|^2\,dt < \infty.
```
:::

En este caso, la suma de un número creciente de armónicos permite aproximar progresivamente la señal original. Si se considera la aproximación parcial de orden $N$,
```{math}
	\tilde{x}_N(t) = \sum_{k=-N}^{N} c_k\,e^{jk\omega_0 t},
```

el error cuadrático medio entre la señal y su representación espectral,
```{math}
	\varepsilon_N = \frac{1}{T_0}\int_{\langle T_0 \rangle} \bigl|\tilde{x}(t)-\tilde{x}_N(t)\bigr|^2\,dt,
```
disminuye al aumentar el número de términos considerados en la serie, de forma que
```{math}
	\lim_{N\to\infty} \varepsilon_N = 0.
```

Este resultado pone de manifiesto que la serie de Fourier reproduce correctamente el comportamiento global de la señal y su contenido espectral, aunque la convergencia no tiene por qué ser puntual en todos los instantes de tiempo.

En señales que presentan discontinuidades pueden aparecer oscilaciones locales en las proximidades de dichas discontinuidades, fenómeno conocido como fenómeno de Gibbs, que no desaparece al aumentar el número de armónicos, pero queda confinado a un entorno reducido alrededor de los puntos de discontinuidad (ver [Efecto de Gibbs](#T3:SF:Gibbs)).

Bajo condiciones adicionales de regularidad, conocidas como condiciones de Dirichlet, puede garantizarse además la convergencia puntual de la serie de Fourier. En particular, dichas condiciones establecen que:

:::{important .simple icon=false} Condiciones de Dirichlet
* La señal es absolutamente integrable en un período,
```{math}
    \frac{1}{T_0}\int_{\langle T_0 \rangle} |\tilde{x}(t)|\,dt < \infty.
```
- La señal presenta un número finito de máximos y mínimos en cada período.
- La señal presenta un número finito de discontinuidades de salto finito en cada período.
:::

En este caso, la serie de Fourier converge al valor de la señal en los puntos de continuidad y al valor medio de los límites laterales en los puntos de discontinuidad.


:::{warning} Efecto de Gibbs
:name: T3:SF:Gibbs
Cuando la señal presenta discontinuidades, se puede observar cómo la aproximación oscila alrededor de la discontinuidad.

Al aumentar el número de armónicos sumados, $N$:
- La amplitud de la oscilación no disminuye.
- La oscilación se confina cada vez en un entorno más pequeño de la discontinuidad.
- La energía de la oscilación disminuye.
:::

In [2]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Slider, BoxAnnotation, Div, Arrow, NormalHead, Label, LabelSet, Span
from bokeh.layouts import column, row
from myst_nb import glue

output_notebook()

# ==============================================================================
# 1. DATOS PRE-CALCULADOS (Periodo T=2)
# ==============================================================================
# Aumentamos un poco la resolución a 5000 para que la vertical sea perfecta
t = np.linspace(-1.0, 2.5, 5000)
# Señal ideal: signo de seno(pi * t) para T=2
square_wave_ideal = np.sign(np.sin(np.pi * t))

def fourier_square_sum(t, n_harmonics):
    approximation = np.zeros_like(t)
    for k in range(1, n_harmonics + 1, 2):
        approximation += np.sin(k * np.pi * t) / k
    approximation *= (4 / np.pi)
    return approximation

N_values = [1, 3, 5, 7, 9, 11, 13, 15, 19, 25, 35, 51, 75, 101, 151, 201, 251]

# Estructuras de datos
data_dict = {'t': t, 'ideal': square_wave_ideal}
peak_data = {} 
zoom_limits = {} 

for n in N_values:
    y_vals = fourier_square_sum(t, n)
    data_dict[f'y_{n}'] = y_vals
    
    # --- CÁLCULO DEL PICO ---
    search_indices = np.where((t > 0) & (t < 0.5))[0] 
    local_y = y_vals[search_indices]
    
    if len(local_y) > 0:
        max_idx_local = np.argmax(local_y)
        max_val = local_y[max_idx_local]
        max_t = t[search_indices[max_idx_local]]
        overshoot_pct = (max_val - 1.0) * 100 / 2
        label_text = f"Max: {max_val:.3f} (+{overshoot_pct:.1f}%)"
    else:
        max_t = 0.5; max_val = 4/np.pi; label_text = f"Max: {max_val:.3f}"

    peak_data[f'p_{n}'] = {'t': [max_t], 'y': [max_val], 'text': [label_text]}
    
    # --- LÍMITES ZOOM ---
    if n < 9:
        zoom_limits[f'z_{n}'] = {'xs': -0.2, 'xe': max_t + 0.2, 'ys': -0.2, 'ye': 1.5}
    else:
        zoom_limits[f'z_{n}'] = {'xs': -0.05, 'xe': 0.2, 'ys': 0.8, 'ye': 1.25}

source = ColumnDataSource(data=dict(t=t, ideal=square_wave_ideal, y=data_dict['y_1']))
source_peak = ColumnDataSource(data=peak_data['p_1'])

# ==============================================================================
# 2. CONFIGURACIÓN VISUAL
# ==============================================================================
COLOR_APROX = 'blue'
X_LIMIT_DRAW = 2.4 
X_LIMITS_AXIS = (-1.0, X_LIMIT_DRAW) 

# --- GRÁFICA PRINCIPAL (ANCHO 450) ---
p_main = figure(height=400, width=450, tools="pan,wheel_zoom,reset",
                x_range=(-1.0, X_LIMIT_DRAW + 0.2), 
                y_range=(-1.6, 2.0), title="Aproximación parcial de la serie de Fourier")

p_main.outline_line_color = None; p_main.grid.visible = False; p_main.axis.visible = False

# Ejes
p_main.segment(x0=X_LIMITS_AXIS[0], y0=0, x1=X_LIMIT_DRAW, y1=0, line_color="black", line_width=1)
p_main.add_layout(Arrow(end=NormalHead(fill_color="black", size=10), 
                        x_start=X_LIMIT_DRAW, y_start=0, x_end=X_LIMIT_DRAW + 0.1, y_end=0))
p_main.add_layout(Label(x=X_LIMIT_DRAW, y=0.1, text="t", text_font_style="italic", text_font_size="12pt"))

p_main.segment(x0=0, y0=-1.5, x1=0, y1=1.8, line_color="black", line_width=1)
p_main.add_layout(Arrow(end=NormalHead(fill_color="black", size=10), x_start=0, y_start=1.8, x_end=0, y_end=1.9))
p_main.add_layout(Label(x=0.1, y=1.8, text=r"$$\tilde{x}_N(t)$$", text_font_size="11pt"))

# Ticks
for i in [1, 2]:
    p_main.add_layout(Label(x=i, y=-0.2, text=str(i), text_align="center", text_font_size="10pt"))
for j in [-1, 1]:
    p_main.add_layout(Label(x=-0.15, y=j, text=str(j), text_align="right", text_baseline="middle", text_font_size="10pt"))
p_main.add_layout(Label(x=-0.1, y=-0.2, text="0", text_font_size="10pt"))

# --- FÓRMULA DINÁMICA (FUENTE 9pt) ---
N_init = N_values[0]
latex_text_init = r"$$\tilde{x}_{" + str(N_init) + r"}(t) = \sum_{k=-" + str(N_init) + r"}^{" + str(N_init) + r"} c_k e^{jk\pi t}$$"
formula_label = Label(x=0.6, y=1.3, text=latex_text_init, text_font_size="9pt", text_color="#333")
p_main.add_layout(formula_label)

# --- CURVAS (PRINCIPAL) ---
p_main.line('t', 'ideal', source=source, color="black", alpha=0.3, line_width=1.5, line_dash="dashed", legend_label="Ideal")
p_main.line('t', 'y', source=source, color=COLOR_APROX, line_width=2, legend_label="Aprox")

# Sombreado Error (Último para la leyenda)
p_main.varea(x='t', y1='ideal', y2='y', source=source, 
             fill_color="red", fill_alpha=0.2, legend_label="Error")

init_zoom = zoom_limits['z_1']
zoom_box = BoxAnnotation(left=init_zoom['xs'], right=init_zoom['xe'], bottom=init_zoom['ys'], top=init_zoom['ye'], 
                         fill_color="gray", fill_alpha=0.1, line_color="gray", line_dash="dotted")
p_main.add_layout(zoom_box)
p_main.legend.location = "top_right"; p_main.legend.click_policy = "hide"; p_main.legend.border_line_color = None

# --- GRÁFICA ZOOM (ANCHO 200) ---
p_zoom = figure(title="Zoom Dinámico", height=400, width=200, 
                x_range=(init_zoom['xs'], init_zoom['xe']), 
                y_range=(init_zoom['ys'], init_zoom['ye']), 
                tools="", toolbar_location=None,
                background_fill_color="white", border_fill_color="white")

# CORRECCIÓN AQUÍ: Quitamos el Span infinito y dibujamos la señal ideal real
# Esto crea el escalón vertical en t=0 y limita la línea a t>0 (donde es 1) y t<0 (donde es -1)
p_zoom.line('t', 'ideal', source=source, color="black", alpha=0.3, line_width=1.5, line_dash="dashed")

p_zoom.varea(x='t', y1='ideal', y2='y', source=source, fill_color="red", fill_alpha=0.2)
p_zoom.line('t', 'y', source=source, color=COLOR_APROX, line_width=3)
p_zoom.scatter('t', 'y', source=source_peak, size=8, color="red", marker="circle")
labels = LabelSet(x='t', y='y', text='text', source=source_peak, 
                  text_font_size="8pt", text_color="red", x_offset=-60, y_offset=5, text_align="left")
p_zoom.add_layout(labels)
p_zoom.xaxis.axis_label = "t"; p_zoom.grid.grid_line_color = "#eeeeee"

# ==============================================================================
# 3. INTERACTIVIDAD
# ==============================================================================
div_info = Div(text=f"<h3 style='margin-bottom:0px; color:#333'>Armónicos: N = {N_values[0]}</h3>", width=300)
slider = Slider(start=0, end=len(N_values)-1, value=0, step=1, title="", sizing_mode="stretch_width", show_value=False) 

callback = CustomJS(args=dict(source=source, source_peak=source_peak, slider=slider, div=div_info, 
                              all_data=data_dict, all_peaks=peak_data, all_lims=zoom_limits, 
                              N_vals=N_values, p_zoom=p_zoom, zoom_box=zoom_box,
                              formula_label=formula_label), code="""
    const idx = slider.value;
    const N = N_vals[idx];
    const data = source.data;
    const y_new = all_data['y_' + N];
    for (let i = 0; i < data['y'].length; i++) { data['y'][i] = y_new[i]; }
    
    const peak_curr = all_peaks['p_' + N];
    source_peak.data = { 't': peak_curr['t'], 'y': peak_curr['y'], 'text': peak_curr['text'] };
    
    const lims = all_lims['z_' + N];
    p_zoom.x_range.start = lims.xs; p_zoom.x_range.end = lims.xe;
    p_zoom.y_range.start = lims.ys; p_zoom.y_range.end = lims.ye;
    zoom_box.left = lims.xs; zoom_box.right = lims.xe; zoom_box.bottom = lims.ys; zoom_box.top = lims.ye;
    
    div.text = "<h3 style='margin-bottom:0px; color:#333'>Armónicos: N = " + N + "</h3>";
    
    formula_label.text = "$$\\\\tilde{x}_{" + N + "}(t) = \\\\sum_{k=-" + N + "}^{" + N + "} c_k e^{jk\\\\pi t}$$";

    source.change.emit(); source_peak.change.emit();
""")

slider.js_on_change('value', callback)

# ==========================================
# CAPTION Y LAYOUT
# ==========================================
texto_caption = """
<div style="font-family: sans-serif; margin-top: 10px; font-size: 14px; opacity: 0.8;">
    <b>Efecto de Gibbs.</b> A medida que aumentamos el número de armónicos <em>N</em> en la aproximación de la serie de Fourier
    de una onda cuadrada, observamos que cerca de los puntos de discontinuidad (t=0, t=1, etc.) aparece un sobreimpulso que no desaparece aunque
    incrementemos <em>N</em>. Este fenómeno, conocido como <em>efecto de Gibbs</em>, resulta en oscilaciones persistentes alrededor del valor ideal,
    alcanzando aproximadamente un 9% por encima del valor esperado. Aunque la aproximación mejora en otras regiones, estas oscilaciones cerca de las discontinuidades son inevitables.
</div>
"""
caption = Div(text=texto_caption, sizing_mode="stretch_width")

layout_final = column(
    div_info,
    slider,
    row(p_main, p_zoom, sizing_mode="scale_width"), 
    caption,
    sizing_mode="scale_width"
)

show(layout_final)

Loading BokehJS ...